In [9]:
import torch
try:
  import torch_geometric
except:
  !pip -q install torch-scatter     -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
  !pip -q install torch-sparse      -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
  !pip -q install torch-cluster     -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
  !pip -q install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.8.0+cu101.html
  !pip -q install torch-geometric
  import torch_geometric
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from torch_geometric.utils.convert import to_networkx
from torch_geometric.datasets import TUDataset
from torch_geometric.data import Data
from scipy.sparse.csgraph import shortest_path
from scipy.sparse import csr_matrix
from torch_geometric.utils.random import erdos_renyi_graph
import time
import random
from math import floor, ceil
from copy import deepcopy
import importlib
from os import path
import bz2
import pickle
import _pickle as cPickle
import sys

from google.colab import drive
folder = '/content/drive/MyDrive/USI/GDLProject/'
drive.mount('/content/drive/', force_remount=True)
 
sys.path.append(folder)
 

import syntheticGraph
import syntheticGraphDataset
import reducedGraph
import GINConv
import GNN
import graph

importlib.reload(syntheticGraph)
importlib.reload(syntheticGraphDataset)
importlib.reload(reducedGraph)
importlib.reload(GINConv)
importlib.reload(GNN)
importlib.reload(graph)

from graph import Graph, Node
from syntheticGraph import syntheticGraph
from syntheticGraphDataset import syntheticGraphDataset
from reducedGraph import reducedGraph
from GINConv import GINConv
from GNN import GNN
 
 
torch.set_default_tensor_type(torch.FloatTensor)

Mounted at /content/drive/


In [10]:
loss_dict = dict()
loss_avg = []
test_loss = []
losses = []
 
hyperparams = {'n_eig' : 40}

In [11]:
folder = '/content/drive/MyDrive/USI/GDLProject/'
reduction_ratio_list = [0.3, 0.5, 0.7]
name_graph_class_list = ['barabasi_albert_graph' ,'erdos_renyi_graph']
reducution_type_list = ['hem','baseline']
name_list = ["training_set", "validation_set", "test_set"]
sizes = {"training_set": [i for i in range(512, 1012, 100)],
         "validation_set":[i for i in range(1012, 1512, 100)],
         "test_set":[i for i in range(1512, 2913, 100)]}
preprocess = True

In [ ]:
for reduction_ratio in reduction_ratio_list:
  for name_graph_class in name_graph_class_list:
    for reducution_type in reducution_type_list:
      for name in name_list:

        dataset_name = folder+name+'_'+str(reduction_ratio)+"_"+name_graph_class+'_'+reducution_type+'_'+'.pbz2'
        start = time.time()
        print("Creating the " +dataset_name +" ...")
        dataset = syntheticGraphDataset(entireMatrix=False)
        for key, s in enumerate(sizes[name]):
          print("\t Creating graph ", key, " of size ", s)
          graph = syntheticGraph(name_graph_class=name_graph_class, size = s,  p = (0.1*512)/s)
          red_graph = reducedGraph(graph, coarse_type=reducution_type, n_supernodes=int(s*reduction_ratio))
          dataset.addGraph(graph, red_graph, hyperparams['n_eig'], preprocessData = preprocess)
        
        syntheticGraphDataset.export_dataset(dataset_name, dataset)
        print(dataset_name+" created in ", time.time()-start, "seconds\n")

Creating the /content/drive/MyDrive/USI/GDLProject/training_set_0.3_barabasi_albert_graph_hem_.pbz2 ...
	 Creating graph  0  of size  512
	 Creating graph  1  of size  612
	 Creating graph  2  of size  712
	 Creating graph  3  of size  812
	 Creating graph  4  of size  912
/content/drive/MyDrive/USI/GDLProject/training_set_0.3_barabasi_albert_graph_hem_.pbz2 created in  21.28483772277832 seconds

Creating the /content/drive/MyDrive/USI/GDLProject/validation_set_0.3_barabasi_albert_graph_hem_.pbz2 ...
	 Creating graph  0  of size  1012
	 Creating graph  1  of size  1112
	 Creating graph  2  of size  1212
	 Creating graph  3  of size  1312
	 Creating graph  4  of size  1412
/content/drive/MyDrive/USI/GDLProject/validation_set_0.3_barabasi_albert_graph_hem_.pbz2 created in  56.10973072052002 seconds

Creating the /content/drive/MyDrive/USI/GDLProject/test_set_0.3_barabasi_albert_graph_hem_.pbz2 ...
	 Creating graph  0  of size  1512
	 Creating graph  1  of size  1612
	 Creating graph  2  